<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Aug 31 06:14:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!git clone https://github.com/w-okada/voice-changer.git
%cd voice-changer/demo/


Cloning into 'voice-changer'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 266 (delta 123), reused 194 (delta 65), pack-reused 0
Receiving objects: 100% (266/266), 19.11 MiB | 35.44 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/voice-changer/demo


In [5]:
!git checkout dev


Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [6]:
!mkdir -p ../frontend/dist
!cp -r ../docs/* ../frontend/dist/
!ls ../frontend/dist


assets	  coffee.png   index.html  index.js.LICENSE.txt
audiolet  favicon.ico  index.js


In [11]:
!cp ../template/setting_colab.json ../frontend/dist/assets/setting.json

In [12]:
!cat ../frontend/dist/assets/setting.json

{
    "app_title": "voice-changer",
    "majar_mode": "colab",
    "voice_changer_server_url": "http://localhost:8080/test",
    "sample_rate": 48000,
    "buffer_size": 1024,
    "prefix_chunk_size": 24,
    "chunk_size": 24,
    "speaker_ids": [100, 107, 101, 102, 103],
    "speaker_names": ["ずんだもん", "user", "そら", "めたん", "つぐみ"],
    "src_id": 107,
    "dst_id": 100,
    "vf_enable": true,
    "voice_changer_mode": "realtime",
    "gpu": 0,
    "available_gpus": [-1, 0, 1, 2, 3, 4],
    "avatar": {
        "motion_capture_face": true,
        "motion_capture_upperbody": true,
        "lip_overwrite_with_voice": true,
        "avatar_url": "./assets/vrm/zundamon/zundamon.vrm",
        "backgournd_image_url": "./assets/images/bg_natural_sougen.jpg",
        "background_color": "#0000dd",
        "chroma_key": "#0000dd",
        "avatar_canvas_size": [1280, 720],
        "screen_canvas_size": [1280, 720]
    },
    "advance": {
        "avatar_draw_skip_rate": 3,
        "screen_draw_ski

# 手作業

・configとモデルをdemoフォルダにコピー

・docsをfrontendに変更

・setting.jsonをfrontendにコピー


In [13]:
!apt-get install -y espeak libsndfile1-dev
!pip install flask
!pip install python-socketio
!pip install eventlet
!pip install unidecode
!pip install phonemizer
!pip install retry
!pip install flask
!pip install flask_cors


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.co

In [14]:
get_ipython().system_raw(f'python3 serverFlask.py 8082 {CONFIG} {MODEL} >foo 2>&1 &')

In [18]:
!cat foo

[2022-08-31 06:17:58,669] INFO in serverFlask: INITIALIZE MODEL
[2022-08-31 06:18:08,764] INFO in utils: Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)


In [27]:
from google.colab import output

output.serve_kernel_port_as_window(8082)

<IPython.core.display.Javascript object>

In [23]:
! ls ../frontend/dist/index.html

../frontend/dist/index.html
